In [65]:
import telegram
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import io
import logging
import pandas as pd
import pandahouse
import os
import datetime as dt

sns.set(rc={'figure.figsize':(19, 7), 'axes.titlesize':16, 'axes.labelsize':14, 'xtick.labelsize':12, 
            'ytick.labelsize':12,'axes.titlepad': 30 })

connection = {
    'host': 'https://clickhouse.lab.karpov.courses',
    'password': 'dpo_python_2020',
    'user': 'student',
    'database': 'simulator_20220220'
}

In [18]:
bot = telegram.Bot(token = '5291500178:AAFQqAVPSG4ad90QdyE-IaZH-Ai8ow2oxFs')
updates = bot.getUpdates()
print(updates[-1])

{'message': {'new_chat_photo': [], 'entities': [], 'caption_entities': [], 'date': 1647777380, 'photo': [], 'new_chat_members': [], 'group_chat_created': False, 'text': ';l;', 'supergroup_chat_created': False, 'chat': {'first_name': 'Никита', 'type': 'private', 'id': 1295860693, 'last_name': 'Иванов'}, 'delete_chat_photo': False, 'message_id': 29, 'channel_chat_created': False, 'from': {'id': 1295860693, 'last_name': 'Иванов', 'language_code': 'ru', 'is_bot': False, 'first_name': 'Никита'}}, 'update_id': 807764393}


In [19]:
chat_id = 1295860693

In [20]:
q = 'SELECT toDate(time) as date, user_id, action FROM {db}.feed_actions where toDate(time) > today() -7'

df = pandahouse.read_clickhouse(q, connection=connection)

In [34]:
DAU = df.groupby('date', as_index = False).user_id.nunique().rename(columns={'user_id':'DAU'})
Likes = df.loc[df.action == 'like'].groupby('date', as_index = False).user_id.count().rename(columns={'user_id':'likes'})
Views = df.loc[df.action == 'view'].groupby('date', as_index = False).user_id.count().rename(columns={'user_id':'views'})

In [37]:
report_metrics = DAU.merge(Likes).merge(Views)

In [38]:
report_metrics['CTR_%'] = round(report_metrics.likes*100/report_metrics.views,2)

In [60]:
report_metrics

,date,DAU,likes,views,CTR_%
0,2022-03-14,19072,119012,574585,20.71
1,2022-03-15,19211,135958,675874,20.12
2,2022-03-16,20316,113263,583467,19.41
3,2022-03-17,20187,100663,522682,19.26
4,2022-03-18,20584,97989,502954,19.48
5,2022-03-19,20322,123776,583300,21.22
6,2022-03-20,15438,80723,371075,21.75


In [71]:
name_metrics = report_metrics.drop('date',1).columns.values
value_metrics = report_metrics.iloc[-2,:].values
# send message to telegram
message = 'Отчет за {v0} по значениям продуктовых метрик:\n-DAU {v1},\n-количество лайков {v2},\n-количество просмотров {v3},\n-CTR {v4}%.'\
        .format(v0 = value_metrics[0].date(), v1 = value_metrics[1], v2 = value_metrics[2], v3 = value_metrics[3], v4 = value_metrics[4])
bot.sendMessage(chat_id=chat_id, text=message)

/tmp/ipykernel_48/3584072339.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  name_metrics = report_metrics.drop('date',1).columns.values


In [80]:
for metrics in name_metrics:
    sns.lineplot(x='date', y= metrics, data = report_metrics)
    plt.title('{v}'.format(v=metrics))
    plot_object = io.BytesIO()
    plt.savefig(plot_object)
    plot_object.seek(0)
    plot_object.name = '{v}.png'.format(v=metrics)
    plt.close()
    bot.sendPhoto(chat_id=chat_id, photo=plot_object)

In [81]:
file_object = io.StringIO()
report_metrics.to_csv(file_object)
file_object.name = 'report_metrics.csv'
file_object.seek(0)
bot.sendDocument(chat_id=chat_id, document=file_object)